In [33]:
from src import *
import torch
from torch import nn
import numpy
import numpy as np

In [34]:
x=torch.rand([2, 3, 10, 10])
decimal_to_bits(x, bits=8).shape
X = (x * 255).int().clamp(0, 255)


In [35]:
def decimal_to_qubits(x,bits=BITS):
    """
        makes the wavefunction collapse
        expects image tensor with each x representing the polar angle of each qbit
    """
    device = x.device

    x = (x * 255).int().clamp(0, 255)

    # powers of 2 [128,  64,  32,  16,   8,   4,   2,   1]
    mask = 2 ** torch.arange(bits - 1, -1, -1, device = device)
    mask = rearrange(mask, 'd -> d 1 1') #mask.shape == (8, 1, 1)
    x = rearrange(x, 'b c h w -> b c 1 h w') #batch, channel, height, width

    bits = ((x & mask) != 0).float()*np.pi/2
    bits = rearrange(bits, 'b c d h w -> b (c d) h w')
    return bits

def qubit_to_decimal(x, bits = BITS):
    """ expects x to be a polar angle in randians, returns a decimal number in the range [0, 1] """
    device = x.device

    x = torch.bernoulli(torch.sin(x)**2).int()
    mask = 2 ** torch.arange(bits - 1, -1, -1, device = device, dtype = torch.int32)

    mask = rearrange(mask, 'd -> d 1 1')
    x = rearrange(x, 'b (c d) h w -> b c d h w', d = 8)
    dec = reduce(x * mask, 'b c d h w -> b c h w', 'sum')
    return (dec / 255).clamp(0., 1.)